In [ ]:
from typing import List, Dict, Optional
from uuid import uuid4
from enum import Enum

class MObjectType(str, Enum):
    REQUIREMENT = "Requirement"
    TASK = "Task"
    DOCUMENT = "Document"
    COMPONENT = "Component"

class MObject:
    def __init__(self, name: str, obj_type: MObjectType, properties: Dict = {}):
        self.id = str(uuid4())  # Unique ID for traceability
        self.name = name
        self.obj_type = obj_type
        self.properties = properties or {}

        # Relationship links
        self.satisfies: List[str] = []       # Upstream links (to higher-level requirements)
        self.satisfied_by: List[str] = []    # Downstream links (to lower-level requirements)
        self.verifies: List[str] = []     # Linked verification activities
        self.verified_by: List[str] = []    # Linked validation activities

    def add_satisfies(self, other: 'MObject'):
        if other.id not in self.satisfies:
            self.satisfies.append(other.id)

    def add_satisfied_by(self, other: 'MObject'):
        if other.id not in self.satisfied_by:
            self.satisfied_by.append(other.id)

    def add_verified_by(self, other: 'MObject'):
        if other.id not in self.verified_by:
            self.verified_by.append(other.id)

    def add_validated_by(self, other: 'MObject'):
        if other.id not in self.validated_by:
            self.validated_by.append(other.id)

    def to_dict(self):
        return {
            "id": self.id,
            "name": self.name,
            "obj_type": self.obj_type.value,
            "properties": self.properties,
            "satisfies": self.satisfies,
            "satisfied_by": self.satisfied_by,
            "verified_by": self.verified_by,
            "validated_by": self.validated_by,
        }

class RelationshipType(str, Enum):
    SATISFIES = "satisfies"
    SATISFIED_BY = "satisfied_by"
    VERIFIED_BY = "verified_by"
    VALIDATED_BY = "validated_by"

class MObjectProperty(str, Enum):
    MATRIX_SIZE = "Matrix_Size"
    CORE_TECH = "Polatis_DirectLight_beam-steering"
    INTERFACES = "Interfaces"
    OPTIONAL_FEATURES = "Optional_Features"
    OPTICAL_TYPE = "Single_Mode"

    MAX_IL_DB = "Max_Insertion_Loss_dB"
    MAX_RL_DB = "Max_Return_Loss_dB"
    MAX_SWITCHING_TIME_MS = "Max_Switching_Time_ms"
    CONNECTION_STABILITY_DB = "Repeatability_dB"
    WAVELENGTH_RANGE_NM = "Wavelength_Range_nm"
    LATENCY_NS = "Latency_ns"
    WDL_DB = "Wavelength_Dependent_Loss_dB"
    PDL_DB = "Polarization_Dependent_Loss_dB"
    CROSSTALK_DB = "Crosstalk_dB"
    OPTICAL_INPUT_POWER_DBM = "Optical_Input_Power_dBm"
    SWITCH_LIFETIME_CYCLES = "Switch_Lifetime_dB"
    RACK_WIDTH_INCHES = "Rack_Width_Inches"
    INTERFACES = "Interfaces"
    
    REQTYPE_FUNCTIONAL = "Functional"
    REQTYPE_PERFORMANCE = "Performance"
    REQTYPE_INTERFACE = "Interface"
    REQTYPE_RELIABILITY = "Reliability"
    REQTYPE_MAINTAINABILITY = "Maintainability"
    REQTYPE_SAFETY = "Safety"
    REQTYPE_SECURITY = "Security"
    REQTYPE_OTHER = "Other"
    CTRL_INTERFACES = "Control_Interfaces"
    POWER_SUPPLY = "Power_Supply"

    VERIFICATION_METHOD_ANALYSIS = "Analysis"
    VERIFICATION_METHOD_DEMONSTRATION = "Demonstration"
    VERIFICATION_METHOD_INSPECTION = "Inspection"
    VERIFICATION_METHOD_TEST = "Test"

class MObjectManager:
    def __init__(self):
        self.objects: Dict[str, MObject] = {}

    def add(self, obj: MObject):
        self.objects[obj.id] = obj

    def get(self, obj_id: str) -> Optional[MObject]:
        return self.objects.get(obj_id)

    def find_by_type(self, obj_type: MObjectType) -> List[MObject]:
        return [obj for obj in self.objects.values() if obj.obj_type == obj_type]

    def link_satisfies(self, source_id: str, target_id: str):
        if source_id in self.objects and target_id in self.objects:
            self.objects[source_id].add_satisfies(self.objects[target_id])
            self.objects[target_id].add_satisfied_by(self.objects[source_id])

    def link_verified_by(self, source_id: str, verifier_id: str):
        if source_id in self.objects and verifier_id in self.objects:
            self.objects[source_id].add_verified_by(self.objects[verifier_id])

    def link_validated_by(self, source_id: str, validator_id: str):
        if source_id in self.objects and validator_id in self.objects:
            self.objects[source_id].add_validated_by(self.objects[validator_id])

In [11]:
print([i.name for i in MObjectProperty])

['MATRIX_SIZE', 'CORE_TECH', 'INTERFACES', 'OPTIONAL_FEATURES', 'OPTICAL_TYPE', 'MAX_IL_DB', 'MAX_RL_DB', 'MAX_SWITCHING_TIME_MS', 'CONNECTION_STABILITY', 'WAVELENGTH_RANGE_NM']


In [1]:
import pickle

# Load MObjects
with open("polatis_6000i_mobjects.pkl", "rb") as f:
    mobjects = pickle.load(f)

# Index by ID for fast lookup
mobject_by_id = {obj['id']: obj for obj in mobjects}

# Example: print top-level PRD
prd = next(obj for obj in mobjects if obj['obj_type'] == 'PRD')
print("PRD:", prd['name'])
print("Properties:", prd['properties'])

# Example: list all linked requirements
linked_reqs = [obj for obj in mobjects if prd['id'] in obj.get('satisfies', [])]
print(f"\\nRequirements satisfied by {prd['name']}:")
for req in linked_reqs:
    print("-", req['name'], ":", req['properties'])

PRD: Polatis 6000i PRD
Properties: {'product_name': 'Polatis 6000i', 'matrix_size_range': '8x8 to 192x192', 'technology': 'DirectLight beam-steering', 'control_interfaces': ['NETCONF', 'RESTCONF', 'SNMP', 'TL1', 'SCPI'], 'optional_features': ['OPM', 'VOA', 'port shutter'], 'optical_type': 'single-mode', 'use_cases': ['automated testing', 'quantum networking', 'cloud SDN'], 'performance_category': 'Ultra (for sizes up to 96x96)'}
\nRequirements satisfied by Polatis 6000i PRD:
- Insertion Loss : {'max_loss_db': {'32x32': 1.0, '96x96': 1.2, '192x192': 1.9}}
- Insertion Loss w/ OPM : {'max_loss_db': {'32x32': 1.3, '96x96': 1.5, '192x192': 2.2}}
- Switching Time : {'max_switching_time_ms': 25}
- Connection Stability : {'repeatability_dB': 0.05, 'stability_dB': 0.05}
- Wavelength Range : {'range_nm': '1260-1675'}
- Return Loss : {'min_return_loss_dB': 50, 'connector_type': 'APC'}
- Latency : {'max_data_latency_ns': 25}
- Wavelength Dependent Loss : {'wdl_max_dB': 0.3}
- Polarization Dependen

In [2]:
mobjects

[{'id': '544807a8-0857-4f3e-84d1-82bd3bb4c87c',
  'name': 'Polatis 6000i PRD',
  'obj_type': 'PRD',
  'properties': {'product_name': 'Polatis 6000i',
   'matrix_size_range': '8x8 to 192x192',
   'technology': 'DirectLight beam-steering',
   'control_interfaces': ['NETCONF', 'RESTCONF', 'SNMP', 'TL1', 'SCPI'],
   'optional_features': ['OPM', 'VOA', 'port shutter'],
   'optical_type': 'single-mode',
   'use_cases': ['automated testing', 'quantum networking', 'cloud SDN'],
   'performance_category': 'Ultra (for sizes up to 96x96)'},
  'satisfies': [],
  'satisfied_by': [],
  'verified_by': [],
  'validated_by': []},
 {'id': '02af75d1-f985-4a6c-9aa4-69b2cac0f331',
  'name': 'Insertion Loss',
  'obj_type': 'Requirement',
  'properties': {'max_loss_db': {'32x32': 1.0, '96x96': 1.2, '192x192': 1.9}},
  'satisfies': ['544807a8-0857-4f3e-84d1-82bd3bb4c87c'],
  'satisfied_by': [],
  'verified_by': [],
  'validated_by': []},
 {'id': 'a0205701-196a-4f5b-a879-621f07cfb33d',
  'name': 'Insertion Los